# 2.5. Automatic Differentiation

In [9]:
# https://zhang-yang.medium.com/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29

# -------------------------------------------------------------------------------
# Gradient with: input is vector, output is scalar

import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

Antes de calcularmos o gradiente de y em relação a x, precisamos armazena-lo. É importante que não aloquemos
nova memória cada vez que tomamos uma derivada em relação a um parâmetro porque costumamos atualizar os
mesmos parâmetros milhares ou milhões de vezes e podemos rapidamente ficar sem memória.

In [10]:
# x = torch.tensor([[0., 1., 2., 3.]])
# xt = x.T
# y=2*xt*x

x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
x.grad  # O valor padrão é None

y = 2 * torch.dot(x, x)  #produto escalar (ou produto interno): o resultado é uma grandeza escalar
# O segundo x é transposto ao primeiro x para possibilitar o produto escalar: x*x_T
# 2.3.8. Produto escalar (Dot Products - also known as inner product) - resultado é uma grandeza escalar
# É a soma dos produtos dos elementos na mesma posição
# torch.dot(x, y)    # O que é equivalente a fazer: torch.sum(x * y)

Em seguida, podemos calcular automaticamente o gradiente de y com relação a cada componente de x chamando
a função de retropropagação e imprimindo o gradiente.

In [11]:
y.backward()  #função de retropropagação
x.grad        #imprime o gradiente

x.grad == 4 * x # Verificando se o gradiente foi calculado corretamente

# O PyTorch acumula os gradientes por padrão, precisamos
# apagar os valores anteriores
x.grad.zero_()

tensor([0., 0., 0., 0.])

#  RESUMO 1:

Gradient with: input is vector, output is scalar

In [12]:
import torch

x = torch.arange(4.0) # tensor x
x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
x.grad  # O valor padrão é None
y = 2 * torch.dot(x, x)  # Função desejada
y.backward()  #chama a função de retropropagação
x.grad        #imprime o gradiente
x.grad.zero_() # apagar os valores anteriores

tensor([0., 0., 0., 0.])

#  RESUMO 2:

Gradient with: input is vector, output is vector

In [13]:
import torch

x = torch.arange(4.0) # tensor x
print('x:', x)
x.requires_grad_(True)  # Same as `x = torch.arange(4.0, requires_grad=True)`
# x.grad.zero_()  # Não está dando certo. Comentei e funcionou mesmo assim.
y = 3*x**2
print('y:', y)
# y.backward(torch.ones(len(x))) # torch.ones(len(x)): cria tensor de 1s. # A oção abaixo é mais rápida:
y.sum().backward()
x.grad
print('x.grad:', x.grad)

x: tensor([0., 1., 2., 3.])
y: tensor([ 0.,  3., 12., 27.], grad_fn=<MulBackward0>)
x.grad: tensor([ 0.,  6., 12., 18.])


In [14]:
# Opção 2:
# input is vector, output is vector
import torch
x = torch.tensor([0., 1., 2., 3.], requires_grad=True)
print('x:', x)
y = 3*x**2        # Funciona tb com produto escalar: y = (3*x)@(x**2)
print('y:', y)
gradient_value = [1.]*4  # Lista de 1s com dois valores
y.backward(torch.tensor(gradient_value))
print('x.grad:', x.grad)   # Gradient ou a derivada
# Out:
# x: tensor([1., 2.], requires_grad=True)
# y: tensor([ 3., 12.], grad_fn=<MulBackward0>)
# x.grad: tensor([ 6., 12.])

x: tensor([0., 1., 2., 3.], requires_grad=True)
y: tensor([ 0.,  3., 12., 27.], grad_fn=<MulBackward0>)
x.grad: tensor([ 0.,  6., 12., 18.])


# RESUMO 3: função com 2 variáveis: f(x1,x2)

Gradient with: input is vector, output is vector

In [15]:
import torch
from math import e

x1 = torch.arange(4.0) # tensor x1
x2 = torch.arange(4.0) # tensor x2
print('x1:', x1)
print('x2:', x2)
x1.requires_grad_(True)  # Same as `x1 = torch.arange(4.0, requires_grad=True)`
x2.requires_grad_(True)  # Same as `x2 = torch.arange(4.0, requires_grad=True)`
# x.grad.zero_()  # Não está dando certo. Comentei e funcionou mesmo assim.
y = 3*(x1**2) + 5*(e**x2)  # y = f(x1,x2) --> função de duas variáveis
print('y:', y)
# y.backward(torch.ones(len(x))) # torch.ones(len(x)): cria tensor de 1s. # A oção abaixo é mais rápida:
y.sum().backward()
x1.grad
x2.grad
print('x1.grad:', x1.grad)   # Derivada parcial de y com relação a x1
print('x2.grad:', x2.grad)   # Derivada parcial de y com relação a x2
grad = torch.stack([x1.grad,x2.grad])    # stack=empilhamento:  junção dos dois tensores em um só de 2x4.

x1: tensor([0., 1., 2., 3.])
x2: tensor([0., 1., 2., 3.])
y: tensor([  5.0000,  16.5914,  48.9453, 127.4277], grad_fn=<AddBackward0>)
x1.grad: tensor([ 0.,  6., 12., 18.])
x2.grad: tensor([  5.0000,  13.5914,  36.9453, 100.4277])


Apêndice:

https://autograd.readthedocs.io/en/latest/usage.html

# Derivadas sequenciais:

In [1]:
# Exemplo 1: duas derivações

import torch

x = torch.arange(5.0) # tensor x
print('x:', x)
x.requires_grad_(True)
y = 3*x**2
print('y:', y)

# First derivative:
dy1 = torch.autograd.grad(y, x, grad_outputs=torch.ones(x.shape), retain_graph=True, create_graph=True)
print('dy1:', dy1)

# Second derivative:
dy2 = torch.autograd.grad(dy1, x, grad_outputs=torch.ones(x.shape))  # Para mais uma derivação é preciso adicionar: retain_graph=True, create_graph=True)
print('dy2:', dy2)

x: tensor([0., 1., 2., 3., 4.])
y: tensor([ 0.,  3., 12., 27., 48.], grad_fn=<MulBackward0>)
dy1: (tensor([ 0.,  6., 12., 18., 24.], grad_fn=<MulBackward0>),)
dy2: (tensor([6., 6., 6., 6., 6.]),)


In [2]:
# Exemplo 2: três derivações

import torch

x = torch.arange(5.0) # tensor x
print('x:', x)
x.requires_grad_(True)
y = 2*x**3
print('y:', y)

# First derivative:
dy1 = torch.autograd.grad(y, x, grad_outputs=torch.ones(x.shape), retain_graph=True, create_graph=True)
print('dy1:', dy1)

# Second derivative:
dy2 = torch.autograd.grad(dy1, x, grad_outputs=torch.ones(x.shape), retain_graph=True, create_graph=True)
print('dy2:', dy2)

# Third derivative:
dy3 = torch.autograd.grad(dy2, x, grad_outputs=torch.ones(x.shape), retain_graph=True, create_graph=True)
print('dy3:', dy3)

x: tensor([0., 1., 2., 3., 4.])
y: tensor([  0.,   2.,  16.,  54., 128.], grad_fn=<MulBackward0>)
dy1: (tensor([ 0.,  6., 24., 54., 96.], grad_fn=<MulBackward0>),)
dy2: (tensor([ 0., 12., 24., 36., 48.], grad_fn=<MulBackward0>),)
dy3: (tensor([12., 12., 12., 12., 12.], grad_fn=<MulBackward0>),)
